In [1]:
import pandas as pd
from database import Database
from datetime import datetime
from Filter.Market import Market
import os

In [ ]:
# db = Database()
# q_data=db.get_finance_report_q()
# s_data = db.get_stock('2023/05/16','2023/09/16')
# mk_data = db.get_market_index('2023/05/16','2023/09/16')
# company_data =db.get_company_basic()

In [13]:
DATA_DIR = ''
FILE_NAME = 'data/company_basic1025.csv'
data_path = os.path.join(DATA_DIR, FILE_NAME)
df_from_db = pd.read_csv(data_path)
#TODO:規模何時算?
# df_from_db = df_from_db_o.copy()
df_from_db = df_from_db.sort_values(by='capital', ascending=False)
df_from_db['scale'] = '小'
df_from_db.loc[df_from_db.sort_values(by='capital', ascending=False).index[:50], 'scale'] = '大'
df_from_db.loc[df_from_db.sort_values(by='capital', ascending=False).index[50:150], 'scale'] = '中'
# df_from_db.loc[:50, 'scale'] = '大'
# df_from_db
# df_from_db.loc[50:150, 'scale'] = '中'
# count1 = df_from_db["scale"].value_counts().get('中', 0)
# count1
df_from_db

,company_id,market_type,industry,company_name,industry_name,capital,scale
311,2330,TSE,24,台積電,半導體,2.590000e+11,大
543,2891,TSE,17,中信金,金融業,2.010000e+11,大
535,2883,TSE,17,開發金,金融業,1.840000e+11,大
534,2882,TSE,17,國泰金,金融業,1.620000e+11,大
227,2002,TSE,10,中鋼,鋼鐵工業,1.580000e+11,大
...,...,...,...,...,...,...,...
1393,6419,OTC,26,京晨科,光電業,1.315917e+08,小
636,3095,OTC,27,及成,通信網路業,1.300000e+08,小
1086,5205,OTC,35,中茂,綠能環保,1.025000e+08,小
1754,9110,TSE,91,越南控-DR,存託憑證,3.816800e+07,小
